In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import cv2 as cv
import io
import PIL
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
x_train=[]
y_train=[]
imagesize=150
labels=['glioma','meningioma','notumor','pituitary']

for i in labels:
    folderpath=os.path.join('/kaggle/input/brain-tumor-mri-dataset/Training',i)
    for j in os.listdir(folderpath):
        img=cv.imread(os.path.join(folderpath,j))
        img=cv.resize(img,(imagesize,imagesize))
        x_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderpath=os.path.join('/kaggle/input/brain-tumor-mri-dataset/Testing',i)
    for j in os.listdir(folderpath):
        img=cv.imread(os.path.join(folderpath,j))
        img=cv.resize(img,(imagesize,imagesize))
        x_train.append(img)
        y_train.append(i)
        
x_train=np.array(x_train)
y_train=np.array(y_train)

In [ ]:
k=0
fig, ax = plt.subplots(1,4,figsize=(20,20))
fig.text(s='Sample Image From Each Label',size=28,fontweight='bold',
             fontname='monospace',y=0.62,x=0.4,alpha=0.8)
for i in labels:
    
    j=0
    while True :
        if y_train[j] == i:
            ax[k].imshow(x_train[j])
            ax[k].set_title(y_train[j])
            ax[k].axis('off')
            k+=1
            break
        j+=1

In [ ]:
x_train.shape , y_train.shape

## Splitting the data 

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [ ]:
x_train.shape , x_test.shape , y_train.shape, y_test.shape

In [ ]:
y_train_new=[]
for i in y_train:
    y_train_new.append(labels.index(i))
y_train=y_train_new
y_train=tf.keras.utils.to_categorical(y_train)

y_test_new=[]
for i in y_test:
    y_test_new.append(labels.index(i))
y_test=y_test_new
y_test=tf.keras.utils.to_categorical(y_test)

In [ ]:
y_train[0][2]

In [ ]:
y_test

## Standardizing the dataset (Scalling)

In [ ]:
x_train=x_train/x_train.max()

In [ ]:
x_train.max()

In [ ]:
x_test=x_test/x_test.max()

In [ ]:
x_test.max()

## Building The Model

In [ ]:
tf.random.set_seed(42)

model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(100,(3,3),1, activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(200,(3,3),1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(250,(3,3),1,activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(300,(3,3),1,activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(380,(3,3),1,activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(400,activation='relu'),
    
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])

history=model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test),
                  validation_split=0.1)

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

In [ ]:
pred

## Classification Report

In [ ]:
print(classification_report(y_test_new,pred,target_names=labels))

## Confusion Matrix and Heatmap

In [ ]:
con=confusion_matrix(y_test_new,pred)
con

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(con,cmap='Blues',annot=True,xticklabels=labels,yticklabels=labels,
            linewidths=2)
plt.yticks(rotation='horizontal')
plt.show()

## Making Predictions

In [ ]:
def prediction2(image):
    
    img=cv.imread(image)
    img=cv.resize(img,(150,150))
    img_array=np.array(img)
    
    y_pred=model.predict(np.expand_dims(img_array/255,0))
    y_pred=np.argmax(y_pred,axis=1)
    
    
    if y_pred == 0:
        print(y_pred,'= Giloma tumor')
    elif y_pred == 1:
        print(y_pred,'= Meningioma Tumor')
    elif y_pred == 2:
        print(y_pred,'= No Tumor')
    else:
        print(y_pred,'= Pituitary Tumor')
        
        
    plt.imshow(img)

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0005.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0007.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/meningioma/Te-me_0066.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/notumor/Te-no_0014.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/notumor/Te-no_0013.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/Te-piTr_0009.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/Te-piTr_0007.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/Te-pi_0013.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0002.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0005.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-gl_0011.jpg')

In [ ]:
prediction2('/kaggle/input/brain-tumor-mri-dataset/Testing/meningioma/Te-meTr_0007.jpg')